# Exporting American Movie Box Office Hits 

### Regresssion Model Stepwise Analysis <a id='top'></a> 

1. [Research Question](#1)<br/>
2. [Scraped: Movie Adaptations Data](#2) <br/>
3. [Exporatory Data Analysis: Movie Adaptations Dataframe](#3)<br/>
   [3a. Explore features correlation](#3a)<br/>
   [3b. Explore and handle categorical data](#3b)<br/>
4. [Cross-Validation](#4)<br/>
5. [Modeling](#5)<br/>
   [5a. Simple linear regression model](#5a)<br/>
   [5b. Linear regression models](#5b)<br/>
6. [Model Tuning](#6) <br/>
   [6a. Regularization](#6a)<br/>
   [6b. Features engineering](#6b)<br/>
   [6c. Linear regression assumptions](#6c)<br/>
7. [Best Model ](#7)<br/>
8. [Results](#8)<br/>
   [8a. Interpretability](#8a)<br/>
   [8b. Predictions](#8b)<br/>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt


In [73]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
%matplotlib inline


## 1. Research Question<a id='1'></a> 

* RQ. Can a model predict a movie adaptaion <sup>1</sup> international total gross revenue based on movie data available on boxofficemojo.com?
* Data source: boxofficemojo.com 
* Error metric: adjusted R<sup>2</sup>, MAE, RMSE???? 

<sup>1</sup> Adapted from books, television shows, events, video games, or plays. 


## 2. Scraped [Movie Adaptations Data](https://github.com/slp22/regression-project/blob/main/adaptation_movies_webscraping.ipynb) 


## 3. Exporatory Data Analysis: [Movie Adaptations Dataframe](https://github.com/slp22/regression-project/blob/main/adaptation_movies_eda.ipynb) 

In [64]:
movie_df = pd.read_csv('clean_df.csv')
movie_df.drop(columns=['link_stub'], inplace=True)
movie_df.head(1)

,international_total_gross,domestic_total_gross,rank,movie_title,max_theaters,opening_theathers,worldwide_total_gross,domestic_opening,budget,release_date,runtime,distributor,genres,rating
0,1119261396,543638043,6,The Lion King,4802.0,4725.0,1662899439,191770759,260000000,20190711,118.0,Walt Disney Studios Motion Pictures,Adventure Animation Drama Family Musical,PG


In [ ]:
movie_df.describe()

In [85]:
# check for null values
movie_df.isnull().sum()

international_total_gross                           0
domestic_total_gross                                0
rank                                                0
movie_title                                         0
max_theaters                                        9
opening_theathers                                  18
worldwide_total_gross                               0
domestic_opening                                    0
budget                                              0
release_date                                        0
runtime                                            13
genres                                              0
rating_NC17                                         0
rating_PG                                           0
rating_PG13                                         0
rating_R                                            0
distributor_New Line Cinema                         0
distributor_Paramount Pictures                      0
distributor_Sony Pictures En

In [87]:
# interpolate null values
movie_df['opening_theathers'] = movie_df['opening_theathers'].interpolate()
movie_df['runtime'] = movie_df['runtime'].interpolate()


### 3a. Explore features correlation<a id='3a'></a> 

In [ ]:
# sns.pairplot(movie_df, height=5, aspect=1.5);

In [ ]:
# heatmap correlation matrix
sns.heatmap(movie_df.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

### Correlation Summary

#### Target-Features
*target = `international_total_gross`*
* target correlated with (highest to lowest):
    * `domestic_total_gross`
    * `domestic_opening`
    * `budget`
    * `max_theaters`
    * `opening_theathers`

Target is highly correlated with `worldwide_total_gross`; has known multicollinearity as:<br/>
`worldwide_total_gross` = `domestic_total_gross` + `international_total_gross`

####  Features-Features: Positive Correlation
* domestic_total_gross:
    * `domestic_opening`
    * `worldwide_total_gross`
    * `budget`
    * `max_theaters`
    * `opening_theathers`
* domestic_opening:<br/>
    * `budget`
    * `max_theaters`
    * `opening_theathers`
* max_theaters:
    * `opening_theathers`
    * `budget`
    * `domestic_opening`

####  Features-Features: Negative Correlation
* rank:
    * `domestic_total_gross`
    * `max_theaters`
    * `opening_theathers`
    * `domestic_opening`
    * `budget`
    


### 3b. Explore and handle categorical data<a id='3b'></a> 

In [65]:
# explore genres as candidate for dummies
print('Unique genres:', movie_df.genres.nunique())
print('\n')
print('Genres counts\n', movie_df['genres'].value_counts())

# 👎 Too many for easy-to-use dummy value; let's see what other data works for dummy variables. 

In [66]:
# explore MPAA rating as candidate for dummies
print('Unique MPAA ratings:', movie_df.rating.nunique())
print('\n')
print('Rating counts\n', movie_df['rating'].value_counts())
# 👍 Easy-to-use for dummy variables

In [67]:
# get dummies for MPAA rating 
df_dummies_rating = pd.get_dummies(movie_df, columns=['rating'], drop_first=True)
df_dummies_rating.head(2)
movie_df = df_dummies_rating
movie_df.head(1)

,international_total_gross,domestic_total_gross,rank,movie_title,max_theaters,opening_theathers,worldwide_total_gross,domestic_opening,budget,release_date,runtime,distributor,genres,rating_NC17,rating_PG,rating_PG13,rating_R
0,1119261396,543638043,6,The Lion King,4802.0,4725.0,1662899439,191770759,260000000,20190711,118.0,Walt Disney Studios Motion Pictures,Adventure Animation Drama Family Musical,0,1,0,0


In [68]:
# explore distributor as candidate for dummies 
print('Unique distributors:', movie_df.distributor.nunique())
print('\n')
distributor_count = movie_df['distributor'].value_counts()
print('Distributor count\n', distributor_count)
# 👍 Reasonable amount, group lower frequencies into an other category.

In [69]:
# create distributor other category
distributor_other = list(distributor_count[distributor_count < 20].index)
movie_df['distributor'] = movie_df['distributor'].replace(distributor_other, 'other')

# get dummies for distributor
df_dummies_distributor = pd.get_dummies(movie_df, columns=['distributor'], drop_first=True)
movie_df = df_dummies_distributor
movie_df.head(1)

,international_total_gross,domestic_total_gross,rank,movie_title,max_theaters,opening_theathers,worldwide_total_gross,domestic_opening,budget,release_date,...,rating_PG13,rating_R,distributor_New Line Cinema,distributor_Paramount Pictures,distributor_Sony Pictures Entertainment (SPE),distributor_Twentieth Century Fox,distributor_Universal Pictures,distributor_Walt Disney Studios Motion Pictures,distributor_Warner Bros.,distributor_other
0,1119261396,543638043,6,The Lion King,4802.0,4725.0,1662899439,191770759,260000000,20190711,...,0,0,0,0,0,0,0,1,0,0


In [70]:
movie_df.columns

[back to top](#top)

## 4. Cross-Validation<a id='4'></a> 

In [78]:
# separate target from select features
y = movie_df['international_total_gross']
X = movie_df.loc[:,['domestic_total_gross', 
                    'domestic_opening', 
                    'budget',
                    'rank',
                    'max_theaters', 
                    'opening_theathers',
                    'runtime',
#                     'movie_title', 
                    'rating_NC17', 
                    'rating_PG', 
                    'rating_PG13',
                    'rating_R',
                    'distributor_New Line Cinema',
                    'distributor_Paramount Pictures',
                    'distributor_Sony Pictures Entertainment (SPE)',
                    'distributor_Twentieth Century Fox', 
                    'distributor_Universal Pictures',
                    'distributor_Walt Disney Studios Motion Pictures',
                    'distributor_Warner Bros.', 
                    'distributor_other']]

In [79]:
# split test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
## set up k-folds 
kfold = KFold(n_splits=5, shuffle=True, random_state = 42)

[back to top](#top)

## 5. Modeling<a id='5'></a> 

### 5a. Simple linear regression model<a id='5a'></a> 


In [81]:
# baseline linear regression
lin_reg = LinearRegression()

scores = cross_val_score(lin_reg, X_train, y_train, cv=kfold)
print(scores)
print("Linear regression mean Score: ", np.mean(scores))

lin_reg.fit(X_train, y_train)

[nan nan nan nan nan]
Linear regression mean Score:  nan


/Users/sandraparedes/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/sandraparedes/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/sandraparedes/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_base.py", line 662, in fit
    X, y = self._validate_data(
  File "/Users/sandraparedes/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 581, in _validate_data
    

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# cross-val-score
cross_val_score(lm, X, y, cv=kf, scoring='r2')

In [ ]:
#???????? split train-test
X_train, X_test, y_tes, y_test = train_test_split(movie_df[['domestic_total_gross']], 
                                                movie_df['international_total_gross'], 
                                                test_size=0.20, 
                                                random_state=42)

In [ ]:
#FEATURES: domestic_total_gross
#TARGET: international_total_gross


# simple linear model (slm)
movie_slm = LinearRegression()
movie_slm.fit(X_train,y_train)

print('train score:', round(movie_slm.score(X_train,y_train), 3))
print('test score:', round(movie_slm.score(X_test,y_test), 3))
print('\n')
print('m_1.coef', movie_slm.coef_ )

# model does not perform well

In [ ]:
# residual plot 
f, ax = plt.subplots(figsize=(12, 8))
sns.residplot(x='domestic_total_gross', y='international_total_gross', data=movie_df);

# cone shape indicates heteroskedasticity; likely due to the large range of domestic movie revenue

### 5b. Linear regression models<a id='5b'></a> 
FEATURES: `domestic_total_gross` <br/>
TARGET: `international_total_gross`

In [ ]:
# ## FEAUTRE
# # 'domestic_total_gross'

# ## TARGET
# # 'international_total_gross'

# # y, X = patsy.dmatrices(' ~ domestic_total_gross + domestic_opening + budget + worldwide_total_gross', data=movie_df, return_type="dataframe")
# # y, X = patsy.dmatrices('international_total_gross ~ domestic_total_gross + domestic_opening + budget',                         data=movie_df, return_type="dataframe")
# # y, X = patsy.dmatrices('international_total_gross ~ domestic_total_gross + domestic_opening',                                  data=movie_df, return_type="dataframe")
# # y, X = patsy.dmatrices('international_total_gross ~ domestic_total_gross + budget',                                            data=movie_df, return_type="dataframe")
# y, X = patsy.dmatrices('international_total_gross ~ domestic_total_gross',                                                     data=movie_df, return_type="dataframe")

# # model
# model = sm.OLS(y, X)

# # fit model
# fit = model.fit()

# # model performance statistics
# fit.summary()



In [ ]:
#set up the 3 models we're choosing from:

#1 LIN REG
# 2 RIDGE
# POLY


#1 LIN REG
lm = LinearRegression()

# ________________________________
#feature scaling for train, val, and test so that we can run our ridge model on each
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_val_scaled = scaler.transform(X_val.values)
X_test_scaled = scaler.transform(X_test.values)

# 2 RIDGE
lm_reg = Ridge(alpha=1)

# ________________________________
#feature transforms for train, val, and test so that we can run our poly model on each
poly = PolynomialFeatures(degree=2) 

X_train_poly = poly.fit_transform(X_train.values)
X_val_poly = poly.transform(X_val.values)
X_test_poly = poly.transform(X_test.values)

# 3 POLY
lm_poly = LinearRegression()


In [ ]:
# Fit three models and print R^2 score

#1 LIN REG
lm.fit(X_train, y_train)
print(f'Linear Regression val R^2: {lm.score(X_val, y_val):.3f}')

# 2 RIDGE
lm_reg.fit(X_train_scaled, y_train)
print(f'Ridge Regression val R^2: {lm_reg.score(X_val_scaled, y_val):.3f}')

# 3 POLY
lm_poly.fit(X_train_poly, y_train)
print(f'Degree 2 polynomial regression val R^2: {lm_poly.score(X_val_poly, y_val):.3f}')

[back to top](#top)

## 6. Model Tuning<a id='6'></a> 
√ Take log of some features <br/>
Grid_Search()


In [ ]:
# log transformation for monetary columns 

# check for zeros in columns before log transformation 
count = (movie_df['international_total_gross'] == 0).sum()
print('count of international_total_gross in budget:', count)

count = (movie_df['domestic_total_gross'] == 0).sum()
print('count of domestic_total_gross in budget:', count)

count = (movie_df['budget'] == 0).sum()
print('count of zeros in budget:', count)

count = (movie_df['domestic_opening'] == 0).sum()
print('count of zeros in domestic_opening:', count)

# 👍 international_total_gross and domestic_total_gross

# international_total_gross: min $98, max $1,119,261,000
movie_df['log_international_total_gross'] = np.log(movie_df['international_total_gross'])

# domestic_total_gross: min $742, max $543,638,043
movie_df['log_domestic_total_gross'] = np.log(movie_df['domestic_total_gross'])

# # 👎 zeros throw division-zero error; hold for now 
# # domestic_opening: min $0, max $191,770,800
# movie_df['log_domestic_opening'] = np.log(movie_df['domestic_opening'])

# # budget: min $0, max $270,000,000
# movie_df['log_budget'] = np.log(movie_df['budget'])



In [ ]:
# # scale monetary data to smaller values by dividing by 1,000  or 1,000,000 
### maybe minmaxscalar better?????

# # roounding 
# movie_df['international_total_gross'] = round(movie_df['international_total_gross'].apply(lambda x: x/1000), 2)
# movie_df['domestic_total_gross'] = round(movie_df['domestic_total_gross'].apply(lambda x: x/1000), 2)
# movie_df['worldwide_total_gross'] = round(movie_df['worldwide_total_gross'].apply(lambda x: x/1000), 2)
# movie_df['domestic_opening'] = round(movie_df['domestic_opening'].apply(lambda x: x/1000), 2)
# movie_df['budget'] = round(movie_df['budget'].apply(lambda x: x/1000), 2)
# movie_df['profit'] = round(movie_df['profit'].apply(lambda x: x/1000), 2)

# # not rounding 
# movie_df['international_total_gross'] = movie_df['international_total_gross'].apply(lambda x: x/1000)
# movie_df['domestic_total_gross'] = movie_df['domestic_total_gross'].apply(lambda x: x/1000)
# movie_df['worldwide_total_gross'] = movie_df['worldwide_total_gross'].apply(lambda x: x/1000)
# movie_df['domestic_opening'] = movie_df['domestic_opening'].apply(lambda x: x/1000)
# movie_df['budget'] = movie_df['budget'].apply(lambda x: x/1000)
# movie_df['profit'] = movie_df['profit'].apply(lambda x: x/1000)

# # Code adapated from [sodas32](https://github.com/sodas32/Linear-Regression-Metis-Project-2), thank you!

### 6a. Regularization<a id='6a'></a> 

[regression_lasso_solution](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/02_regression/02_regression_exercises/08-regularization/regression_lasso_solution.ipynb)


In [ ]:
# Standard-scaling features before regularization lines regression

std = StandardScaler()
std.fit(X_train.values)

## apply scaler to train data.

X_train_stdT = std.transform(X_train.values)

## apply scaler to test data

X_test_stdT = std.transform(X_test.values)

In [ ]:
# Grid search to find best parameters to tune model in Lasso, Ridge, and ElasticNet

def build_grid_search_est(model, X, y, cv=5, **params): #(source: regression_lasso_solution.ipynb)

    grid_est = GridSearchCV(model, param_grid=params, cv=kfold, 
                            return_train_score=False)
    grid_est.fit(X, y)
    df = pd.DataFrame(grid_est.cv_results_)
    for param in params:
        df[param] = df.params.apply(lambda val: val[param])
        plt.plot(np.log(df.alpha), df.mean_test_score);
        plt.semilogx(df.alpha, df.mean_test_score)
    return grid_est

In [ ]:
print("Lasso Grid Search") #(source: regression_lasso_solution.ipynb)
lasso_grid_est = build_grid_search_est(Lasso(), X_train, y_train, cv=kfold,
                                       alpha=np.logspace(-4, -1, 30))

In [ ]:
print("Ridge Grid Search") #(source: regression_lasso_solution.ipynb)
ridge_grid_est = build_grid_search_est(Ridge(), X_train, y_train, cv=kfold,
                                       alpha=np.logspace(-4, -1, 10))

In [ ]:
print("Elastic Net Grid Search") #(source: regression_lasso_solution.ipynb)
elastic_net_grid_est = build_grid_search_est(ElasticNet(), X_train, y_train, cv=kfold,
                                             alpha=np.logspace(-4, 0.1, 10))

In [ ]:

params = {
    "alpha": np.logspace(-4, -.1, 20)
}

grid_est = GridSearchCV(Lasso(), param_grid=params, cv=kfold, 
                        return_train_score=False)
grid_est.fit(X_train, y_train)
df = pd.DataFrame(grid_est.cv_results_)
df["alpha"] = df.params.apply(lambda val: val["alpha"])
plt.plot(np.log(df.alpha), df.mean_test_score);

In [ ]:
print("Ridge Model:") #(source: regression_lasso_solution.ipynb)
params = {
    "alpha": np.logspace(-4, -.1, 20)
}

grid_est = GridSearchCV(Ridge(), param_grid=params, cv=kfold, 
                        return_train_score=False)
grid_est.fit(X_train, y_train)
df = pd.DataFrame(grid_est.cv_results_)
df["alpha"] = df.params.apply(lambda val: val["alpha"])
plt.plot(np.log(df.alpha), df.mean_test_score);

In [ ]:
grid_est.best_estimator_

In [ ]:
params['alpha']

In [ ]:
#(source: regression_lasso_solution.ipynb)

y_pred = lin_reg_est.predict(X_holdout)
print("Linear Regression:", r2_score(y_holdout, y_pred))

y_pred = lasso_grid_est.predict(X_holdout)
print("Lasso Regression:", r2_score(y_holdout, y_pred))

y_pred = ridge_grid_est.predict(X_holdout)
print("Ridge Regression:", r2_score(y_holdout, y_pred))

y_pred = elastic_net_grid_est.predict(X_holdout)
print("ElasticNet Regression:", r2_score(y_holdout, y_pred))

In [ ]:
# evaluate model  (source: regression_lasso_solution.ipynb)

# # Fitted vs. Actual
y_train_pred = lin_reg_est.predict(X_train)

# plt.scatter(y_train, y_train_pred, alpha=0.2)
# plt.plot([0, 400], [0, 400])

# # Fitted vs. Actual 
y_test_pred = lin_reg_est.predict(X_holdout)

# plt.scatter(y_holdout, y_test_pred)
# plt.plot([0, 400], [0, 400])

In [ ]:
# # Plot Residuals vs. predicted  (source: regression_lasso_solution.ipynb)

# lin_reg_residuals = y_train - y_train_pred

# plt.scatter(y_train_pred, lin_reg_residuals)
# plt.plot([0,400], [0, 0])
# plt.title("Residuals vs. Predictions")

[back to top](#top)

### 6b. Features engineering<a id='6b'></a> 

In [ ]:
# new feature
# profit = domestic_total_gross - budget

movie_df['profit'] = (movie_df['domestic_total_gross'] - movie_df['budget'])

Code adapated from [sodas32](https://github.com/sodas32/Linear-Regression-Metis-Project-2), thank you!

In [ ]:
# square x5

m_1 = LinearRegression()
m_1.fit(X_train,y_train)

X_train['x5^2'] = X_train['x5']**2 
X_test['x5^2'] = X_test['x5']**2

print('train score with x5^2:', round(m_1.score(X_train,y_train), 3))
print('test score with x5^2:', round(m_1.score(X_test,y_test), 3))
print('\n')
print('m_1.coef  with x5^2', m_1.coef_ )

In [ ]:
# interaction x2 - x3

m_1 = LinearRegression()
m_1.fit(X_train,y_train)

X_train['x2_-_x3'] = (X_train['x2'] - X_train['x3'])
X_test['x2_-_x3'] = (X_test['x2'] - X_test['x3'])


print('train score with x2_-_x3:', round(m_1.score(X_train,y_train), 3))
print('test score with x2_-_x3:', round(m_1.score(X_test,y_test), 3))
print('\n')
print('m_1.coef  with x2_-_x3', m_1.coef_ )

In [ ]:
# interaction x1 * x2

m_3 = LinearRegression()
m_3.fit(X_train,y_train)

X_train['x1_*_x2'] = (X_train['x1']* X_train['x2'])
X_test['x1_*_x2'] = (X_test['x1']* X_test['x2'])


print('train score with x1_*_x2:', round(m_3.score(X_train,y_train), 3))
print('test score with x1_*_x2:', round(m_3.score(X_test,y_test), 3))
print('\n')
print('m_1.coef  with x1_*_x2', m_3.coef_ )

In [ ]:
# deviation 



[back to top](#top)

### 6c. Linear regression assumptions<a id='6c'></a> 
    1. Remove multicollinearity
    2. Transform some features
    3. Look at QQ plots of residuals
    4. Check for independence of errors
    5. Check for heteroskedasticity in residuals!


In [ ]:
# Assumption 1: regression is linear in parameters and correctly specified

# Dx Regression fit: Inspect plot of observed data vs predicted values (points should be symmetric around the line).
# Dx Residual plot: Points should be symmetric around y=0 with roughly constant variance. 
##Look carefully for evidence of a "bowed" pattern, indicating that the model makes systematic errors whenever it is making unusually large or small predictions.
# Q-Q plot: Look for the middle section of dots to be very close to the diagonal red line. 
##Use the chart below as a reference for interpretation.


![](https://i.stack.imgur.com/ZXRkL.png)

In [ ]:
# Assumption 2: residuals ( ${e_i} = Y_i-\hat{Y}_i$ ) should be normally distributed with zero mean
# diagnose/inspect residual normality using qqplot:

stats.probplot(data['resid'], dist="norm", plot=plt)
plt.title("Normal Q-Q plot")
plt.show()

In [ ]:
# Assumption 3: error terms must have constant variance

# create residual plot
movie['predict']=fit.predict(X)
movie['resid']= y-movie.predict
with sns.axes_style('white'):
    plot = movie.plot(
        kind='scatter', x='predict', y='resid', alpha=0.5, figsize=(10,6))
    
# create histogram
movie.DomesticTotalGross.hist();
# note the positive skew

# quick reg plot
plt.scatter(movie.Budget,y)
plt.scatter(movie.Budget,movie.predict);

In [ ]:
# Assumption 4: errors are uncorrelated across observations



In [ ]:
# Assumption 5: no independent variable is a perfect linear function of any other independent variable 
# (no perfect multi-collinearity)

Ways to diagnose: 
1. Inspect correlations of independent features
2. Keep an eye on condition number!
3. As noted above, `statsmodels` will notify you of [large condition numbers](https://en.wikipedia.org/wiki/Condition_number).

Ways to fix:<br/>
3. Consider Partial Least Squares or projection into latent space (PCA, introduced in the second-half of the course)<br/>
4. Use Ridge regularization<br/>

Incorporating [Ridge regularization](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) not only fixes this issue (if it exists, still safe to use if it doesn't), but it imparts other benefits as well. We'll use Ridge quite often after it's officially introduced!

[back to top](#top)

## 7. Best Model<a id='7'></a> 
Fit best model on (train + val), score on test!

In [ ]:
# pd.DataFrame(list(zip(range(10), lasso_grid_est.best_estimator_.coef_)))

[back to top](#top)

## 8. Results<a id='8'></a> 

### 8a. Interpretability<a id='8a'></a> 

In [ ]:
# Coefficients, what are the top predictors

### 8b. Predictions<a id='8b'></a> 

In [ ]:
# Make a prediction for a new value, does it make sense?

In [ ]:
# linear regression plot
f, ax = plt.subplots(figsize=(12, 8))
sns.regplot(x='domestic_total_gross', y='international_total_gross', data=movie_df);


Slides, article, and code available at: https://github.com/slp22/regression-project

[back top top](#top)